Install stable-baselines3 for training algorithms

In [ ]:
!pip install stable_baselines3[extra]

Set-up code

In [ ]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

!pip install gym
!pip install free-mujoco-py

Unzip custom-hopper environment and utils

In [ ]:
!unzip env_images_grayscale.zip

Import libraries

In [ ]:
import gym
from env_images_grayscale.custom_hopper import *
from stable_baselines3 import PPO
# from gym.wrappers.pixel_observation import PixelObservationWrapper
# from gym.wrappers import FrameStack  
# from gym.wrappers import FlattenObservation
# import torch 
# import numpy as np
# import matplotlib.pyplot as plt
# from RawImagesWrapper import *
# from gym.wrappers import FilterObservation
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

Make environment and wrapper to stack frames

In [ ]:
env = gym.make('CustomHopper-source-v0')
vect_env = DummyVecEnv([lambda: gym.make('CustomHopper-source-v0')])
vect_env = VecFrameStack(vect_env, 4)
check_env(env)

Create model and learn

In [ ]:
# policy_kwargs = dict(
#     features_extractor_class=Custom_convLSTM,
#     features_extractor_kwargs=dict(features_dim=512),
# )
model = PPO("CnnPolicy", vect_env, verbose=1, tensorboard_log='./training_logs/grayscale100k')
model.learn(total_timesteps=100000)

Load model and continue learning

In [ ]:
# policy_kwargs = dict(
#     features_extractor_class=Custom_convLSTM,
#     features_extractor_kwargs=dict(features_dim=512),
# )
model = PPO("CnnPolicy", vect_env, verbose=1, tensorboard_log='./drive/MyDrive/training_logs/grayscale200k').load('./drive/MyDrive/trained_models/PPO_grayscale100k', env=vect_env)
model.learn(total_timesteps=100000, progress_bar=True)

Mount drive to save model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Save model for evaluation

In [ ]:
model.save('./drive/MyDrive/trained_models/PPO_grayscale200k')
# model.save('./trained_models/PPO_vect_env')

Evaluate model using evaluation function from stable-baselines3

In [ ]:
mean_rwd, std_rwd = evaluate_policy(model=model, env=vect_env, n_eval_episodes=5, render=False)
print(mean_rwd, std_rwd)